# GDP-16 커스텀 프로젝트 직접 만들기

- 대망의 마지막 project! GDL-15에서 설명했던 HuggingFace transformer 를 이용해 나만의 커스텀 프로젝트를 구성 해 보았습니다.
- 이번 프로젝트에선 mnli 데이터셋을 가지고 나만의 커스텀 프로젝트를 직접 만들어 보겠습니다.
- 마지막인 만큼, 이번 프로젝트는 LMS 클라우드에서 진행합니다.


-----



# 루브릭 평가 기준

| 평가문항 | 상세기준 |
| --- | --- |
| 1. MNLI 데이터셋을 처리하는 전용 Processor 클래스를 정상적으로 구현하였다. | Processor 클래스에 대해 1개 이상의 example에 대한 단위테스트가 정상 진행되었다. |
| 2. BERT tokenizer와 Processor를 결합하여 데이터셋을 정상적으로 생성하였다. | MNLI 데이터셋의 입력과 라벨의 정의에 잘 맞는 tf.data.Dataset 인스턴스가 얻어졌다. |
| 3. MNLI 데이터셋에 대해 적당한 모델을 fine-tuning하여 학습하였다. | 모델 학습이 정상적으로 진행되었다. |

-----


# 1. 필요한 라이브러리 및 데이터셋 불러오기

- 프로젝트 진행에 필요한 라이브러리와 데이터셋을 불러옵니다.
- 설치가 필요한 경우 설치도 함께 진행합니다.



## &nbsp;&nbsp; 1-1 소스 코드 설치하기


- Huggingface와 같은 NLP framework는 해당 framework를 활용하여 새로 만들어진 모델의 성능을 빠르게 평가해 볼 수 있도록 하는 예제 코드를 제공하고 있습니다. 
- 소스코드 프로젝트를 통해 다시 설치 해 봅니다. (클라우드 환경에선 이미 설치되어 있고, 예제때 설치했기 때문에 다시 해 줄 필요는 없습니다.) 
- mnli task는 이전 스텝에서 사용한 BERT를 사용하면 학습이 제대로 되지 않습니다. BERT가 아닌 다른 모델을 선택해야합니다.
- [여기](https://huggingface.co/models) 에서 해당 모델에 대한 task와 모델의 정보 등을 찾아 나만의 프로젝트를 완성 해야합니다... 지만 학습성능이 얼마나 떨어지는지 확인 해봅시다 ㅎ 


In [8]:
# ! cd ~/aiffel && git clone https://github.com/huggingface/transformers.git
# ! cd transformers && pip install -e .
# ! pip install datasets
# ! pip install huggingfac-hub==0.1.0

## &nbsp;&nbsp; 1-2 필요한 라이브러리 불러오기


In [1]:
import os
import numpy as np
from argparse import ArgumentParser
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import BertTokenizer, TFBertForSequenceClassification, AutoConfig
from dataclasses import asdict
from transformers.data.processors.utils import DataProcessor, InputExample, InputFeatures
import transformers
import argparse


## &nbsp;&nbsp; 1-3 데이터셋 다운로드 

- tensorflow-datasets를 이용하여 glue/mnli를 다운로드하려면 tensorflow-datasets 라이브러리 버전을 올려야 합니다.


In [2]:
# 요 명령어를 통해 라이브러리 업그레이드를 진행합니다.
# $ pip install tensorflow-datasets -U

GLUE 데이터셋은 홈페이지에서 원본을 다운로드할 수도 있지만, 이번에는 `tensorflow_datasets`에서 제공하는 것을 이용해 보겠습니다.

아래 코드를 수행해 보면 3668개의 훈련 데이터셋이 존재함을 확인할 수 있을 것입니다.  
mnli 데이터셋을 가져옵니다.

In [3]:
data, info = tfds.load('glue/mnli', with_info=True)
info.splits['train'].num_examples

INFO:absl:Load pre-computed DatasetInfo (eg: splits, num examples,...) from GCS: glue/mnli/2.0.0
INFO:absl:Load dataset info from /tmp/tmphqy4aotttfds
INFO:absl:Field info.release_notes from disk and from code do not match. Keeping the one from code.
INFO:absl:Generating dataset glue (/aiffel/tensorflow_datasets/glue/mnli/2.0.0)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

INFO:absl:Downloading https://dl.fbaipublicfiles.com/glue/data/MNLI.zip into /aiffel/tensorflow_datasets/downloads/dl.fbaipublicfiles.com_glue_MNLIdNe8cK2kTBCG0bqBz2JxwShRT2KfuO3NVIwROTnjtfI.zip.tmp.0a2478ad9e1048c093928c9b253f170c...


...

Generating test_mismatched examples...:   0%|          | 0/9847 [00:00<?, ? examples/s]

Shuffling /aiffel/tensorflow_datasets/glue/mnli/2.0.0.incompleteW1ND71/glue-test_mismatched.tfrecord*...:   0%…

INFO:absl:Done writing /aiffel/tensorflow_datasets/glue/mnli/2.0.0.incompleteW1ND71/glue-test_mismatched.tfrecord*. Number of examples: 9847 (shards: [9847])
INFO:absl:Constructing tf.data.Dataset glue for split None, from /aiffel/tensorflow_datasets/glue/mnli/2.0.0


Dataset glue downloaded and prepared to /aiffel/tensorflow_datasets/glue/mnli/2.0.0. Subsequent calls will reuse this data.


392702

------

# 2.  mnli 데이터셋을 분석해 보기


- 데이터셋을 분석 해 봅니다. 


In [4]:
# data가 어떻게 생겼는지 확인 해 봅니다.
data['train'].take(1)

<TakeDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>

In [8]:
# 어떤 항목이 정의 되어있는지도 확인 해 볼까요?
examples = data['train'].take(1)
for example in examples:
    premise = example['premise']
    hypothesis = example['hypothesis']
    label = example['label']
    print(premise)    
    print(hypothesis)
    print(label)


tf.Tensor(b'In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.', shape=(), dtype=string)
tf.Tensor(b'Meaningful partnerships with stakeholders is crucial.', shape=(), dtype=string)
tf.Tensor(1, shape=(), dtype=int64)


## &nbsp;&nbsp; 2-1 Processor의 활용


- GDL-15 에서 `Huggingface transformers`에서 task별로 데이터셋을 가공하는 일반적인 클래스 구조인 Processor에 대해 다룬 바 있습니다.
- 추상클래스인 Processor를 한번 상속받은 후, Sequence Classification task를 수행하는 모델의 Processor 추상클래스인 DataProcessor 코드 입니다.


In [9]:
class DataProcessor:
    """Base class for data converters for sequence classification data sets."""

    def get_example_from_tensor_dict(self, tensor_dict):
        """
        Gets an example from a dict with tensorflow tensors.

        Args:
            tensor_dict: Keys and values should match the corresponding Glue
                tensorflow_dataset examples.
        """
        raise NotImplementedError()

    def get_train_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the dev set."""
        raise NotImplementedError()

    def get_test_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the test set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    def tfds_map(self, example):
        """
        Some tensorflow_datasets datasets are not formatted the same way the GLUE datasets are. This method converts
        examples to the correct format.
        """
        if len(self.get_labels()) > 1:
            example.label = self.get_labels()[int(example.label)]
        return example

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8-sig") as f:
            return list(csv.reader(f, delimiter="\t", quotechar=quotechar))

print("슝~")

슝~


# 3. MNLIProcessor클래스 구현하기

In [10]:
class MnliProcessor(DataProcessor):
    """Processor for the MRPC data set (GLUE version)."""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def get_example_from_tensor_dict(self, tensor_dict):
        """See base class."""
        return InputExample(
            tensor_dict["idx"].numpy(),
            tensor_dict["premise"].numpy().decode("utf-8"),
            tensor_dict["hypothesis"].numpy().decode("utf-8"),
            str(tensor_dict["label"].numpy()),
        )

    def get_train_examples(self, data_dir):
        """See base class."""
        print("LOOKING AT {}".format(os.path.join(data_dir, "train.tsv")))
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "test.tsv")), "test")

    def get_labels(self):
        """See base class."""
        return ["0", "1", "2"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training, dev and test sets."""
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, i)
            text_a = line[3]
            text_b = line[4]
            label = None if set_type == "test" else line[0]
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples

In [11]:
# get_example_from_tensor_dict() 의 역할은?


processor = MnliProcessor()
examples = data['train'].take(1)

for example in examples:
    print('------원본데이터------')
    print(example)  
    example = processor.get_example_from_tensor_dict(example)
    print('------processor 가공데이터------')
    print(example)



------원본데이터------
{'hypothesis': <tf.Tensor: shape=(), dtype=string, numpy=b'Meaningful partnerships with stakeholders is crucial.'>, 'idx': <tf.Tensor: shape=(), dtype=int32, numpy=16399>, 'label': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'premise': <tf.Tensor: shape=(), dtype=string, numpy=b'In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.'>}
------processor 가공데이터------
InputExample(guid=16399, text_a='In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.', text_b='Meaningful partnerships with stakeh

In [12]:
examples = (data['train'].take(1))
for example in examples:
    example = processor.get_example_from_tensor_dict(example)
    example = processor.tfds_map(example)
    print(example)


InputExample(guid=16399, text_a='In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.', text_b='Meaningful partnerships with stakeholders is crucial.', label='1')


In [14]:
label_list = processor.get_labels()
label_list

['0', '1', '2']

In [15]:
label_map = {label: i for i, label in enumerate(label_list)}

----

# 4. processor 및 Huggingface에서 제공하는 tokenizer를 활용하여 데이터셋 구성하기


- 프레임 워크에서 제공하는 bert 모델과 그에 대응하는 tokenizer를 가져옵니다.
- 미리 훈련된 모델을 가져와 데이터셋을 완성시켜주겠습니다.

In [16]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.





- processor와 tokenizer, 원본 데이터셋을 결합하여 model에 입력할 데이터셋을 생성해 보겠습니다.

In [17]:
def _glue_convert_examples_to_features(examples, tokenizer, max_length, processor, label_list=None, output_mode="claasification") :
    if max_length is None :
        max_length = tokenizer.max_len
    if label_list is None:
        label_list = processor.get_labels()
        print("Using label list %s" % (label_list))

    label_map = {label: i for i, label in enumerate(label_list)}
    labels = [label_map[example.label] for example in examples]

    batch_encoding = tokenizer(
        [(example.text_a, example.text_b) for example in examples],
        max_length=max_length,
        padding="max_length",
        truncation=True,
    )

    features = []
    for i in range(len(examples)):
        inputs = {k: batch_encoding[k][i] for k in batch_encoding}

        feature = InputFeatures(**inputs, label=labels[i])
        features.append(feature)

    for i, example in enumerate(examples[:5]):
        print("*** Example ***")
        print("guid: %s" % (example.guid))
        print("features: %s" % features[i])

    return features

In [18]:
def tf_glue_convert_examples_to_features(examples, tokenizer, max_length, processor, label_list=None, output_mode="classification") :
    """
    :param examples: tf.data.Dataset
    :param tokenizer: pretrained tokenizer
    :param max_length: example의 최대 길이(기본값 : tokenizer의 max_len)
    :param task: GLUE task 이름
    :param label_list: 라벨 리스트
    :param output_mode: "regression" or "classification"

    :return: task에 맞도록 feature가 구성된 tf.data.Dataset
    """
    examples = [processor.tfds_map(processor.get_example_from_tensor_dict(example)) for example in examples]
    features = _glue_convert_examples_to_features(examples, tokenizer, max_length, processor)
    label_type = tf.int64

    def gen():
        for ex in features:
            d = {k: v for k, v in asdict(ex).items() if v is not None}
            label = d.pop("label")
            yield (d, label)

    input_names = ["input_ids"] + tokenizer.model_input_names

    return tf.data.Dataset.from_generator(
        gen,
        ({k: tf.int32 for k in input_names}, label_type),
        ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([])),
    )

print("슝~")

슝~


- `_glue_convert_examples_to_features()` : processor가 생성한 example을 tokenizer로 인코딩하여 feature로 변환하는 역할을 함.  
- 이후 tf_glue_convert_examples_to_features() 함수는 내부적으로
`_glue_convert_examples_to_features()`를 호출해서 얻은 feature를 바탕으로 tf.data.Dataset을 생성하여 리턴합니다.

## &nbsp;&nbsp; 4-1 데이터셋 만들기


In [32]:


max_len = 128
# train_dataset = tf_glue_convert_examples_to_features(data['train'], tokenizer, max_length=max_len, processor=processor)
# examples = train_dataset.take(1)
# for example in examples:
#     print(example)



In [33]:
# train 데이터셋
train_dataset = tf_glue_convert_examples_to_features(data['train'], tokenizer, max_length=max_len, processor=processor)
train_dataset_batch = train_dataset.shuffle(100).batch(16).repeat(2)

Using label list ['0', '1', '2']


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

*** Example ***
guid: 16399
features: InputFeatures(input_ids=[101, 1999, 5038, 1997, 2122, 13136, 1010, 1048, 11020, 2038, 2499, 29454, 29206, 14626, 2144, 2786, 2000, 16636, 1996, 10908, 1997, 1996, 2110, 4041, 6349, 1998, 2000, 5323, 15902, 13797, 2007, 22859, 6461, 2012, 6469, 2075, 1037, 2047, 25353, 14905, 10735, 2483, 2090, 1996, 2976, 10802, 1998, 15991, 1997, 3423, 2578, 4804, 1012, 102, 15902, 13797, 2007, 22859, 2003, 10232, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [34]:
# validation 데이터셋
dataset1 = tf_glue_convert_examples_to_features(data['validation_matched'], tokenizer, max_length=max_len, processor=processor)

#dataset2 = tf_glue_convert_examples_to_features(data['validation_mismatched'], tokenizer, max_length=128, processor=processor)
# validation_dataset = dataset1.concatenate(dataset2)

validation_dataset = dataset1
validation_dataset_batch = validation_dataset.shuffle(100).batch(16)

Using label list ['0', '1', '2']


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

*** Example ***
guid: 6287
features: InputFeatures(input_ids=[101, 7910, 1011, 9616, 2821, 3398, 2035, 1996, 2111, 2005, 2157, 7910, 2166, 2030, 2242, 102, 3398, 7167, 1997, 2111, 2005, 1996, 2157, 2166, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1

In [35]:


# test 데이터셋
dataset3 = tf_glue_convert_examples_to_features(data['test_matched'], tokenizer, max_length=max_len, processor=processor)
# dataset4 = tf_glue_convert_examples_to_features(data['test_mismatched'], tokenizer, max_length=128, processor=processor)

# test_dataset = dataset3.concatenate(dataset4)
test_dataset = dataset3
test_dataset_batch = test_dataset.shuffle(100).batch(16)



Using label list ['0', '1', '2']


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

*** Example ***
guid: 5398
features: InputFeatures(input_ids=[101, 2092, 2003, 2045, 1037, 2210, 3751, 10216, 2017, 2404, 1999, 2045, 102, 2003, 2045, 1037, 10216, 2008, 2017, 2404, 1999, 2045, 1029, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,

----

# 5. 모델 훈련하기 

-  Huggingface의 TFTrainer를 활용해 학습을 진행해 봅니다.
- TFTrainer를 활용하기 위해서는 TFTrainingArguments에 학습 관련 설정을 미리 지정해 두어야 합니다.

In [36]:
# TFTrainer을 활용하는 형태로 모델 재생성
from transformers import (
    AutoConfig,
    AutoTokenizer,
    EvalPrediction,
    HfArgumentParser,
    PreTrainedTokenizer,
    TFAutoModelForSequenceClassification,
    TFTrainer,
    TFTrainingArguments,
    glue_compute_metrics,
    glue_convert_examples_to_features,
    glue_output_modes,
    glue_processors,
    glue_tasks_num_labels,
)

output_dir = os.getenv('HOME')+'/aiffel/transformers'

training_args = TFTrainingArguments(
    output_dir=output_dir,            # output이 저장될 경로
    num_train_epochs=3,              # train 시킬 총 epochs
    per_device_train_batch_size=16,  # 각 device 당 batch size
    per_device_eval_batch_size=64,   # evaluation 시에 batch size
    warmup_steps=500,                # learning rate scheduler에 따른 warmup_step 설정
    weight_decay=0.01,                 # weight decay
    logging_dir='./logs',                 # log가 저장될 경로
    do_train=True,
    do_eval=True,
)

max_seq_length=128
task_name = "mnli"

with training_args.strategy.scope():
    model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    if output_mode == "classification":
        preds = np.argmax(p.predictions, axis=1)
    elif output_mode == "regression":
        preds = np.squeeze(p.predictions)
    return glue_compute_metrics(task_name, preds, p.label_ids)

output_mode = glue_output_modes[task_name]
output_mode



'classification'

In [39]:
train_dataset = train_dataset.apply(tf.data.experimental.assert_cardinality(info.splits['train'].num_examples))
validation_dataset = validation_dataset.apply(tf.data.experimental.assert_cardinality(info.splits['validation_matched'].num_examples))
test_dataset = test_dataset.apply(tf.data.experimental.assert_cardinality(info.splits['test_matched'].num_examples))

In [40]:
trainer = TFTrainer(
    model=model,                           # 학습시킬 model
    args=training_args,                  # TFTrainingArguments을 통해 설정한 arguments
    train_dataset=train_dataset,    # training dataset
    eval_dataset=validation_dataset,       # evaluation dataset
    compute_metrics=compute_metrics,
)

In [41]:
# Training
if training_args.do_train:
    trainer.train()
    trainer.save_model()
    tokenizer.save_pretrained(training_args.output_dir)

TypeError: '>' not supported between instances of 'NoneType' and 'int'

In [42]:
# Evaluation
results = {}
if training_args.do_eval:
    result = trainer.evaluate()
    output_eval_file = os.path.join(training_args.output_dir, "eval_results2.txt")

    with open(output_eval_file, "w") as writer:
        for key, value in result.items():
            writer.write(f"{key} = {value}\n")

        results.update(result)
        
#파일에 쓴 테스트 결과 확인
!cat ~/aiffel/transformers/eval_results2.txt

/opt/conda/lib/python3.9/site-packages/transformers/data/metrics/__init__.py:66: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/opt/conda/lib/python3.9/site-packages/transformers/data/metrics/__init__.py:36: FutureWarning: This metric will be removed from the library soon, metrics should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


eval_loss = nan
eval_mnli/acc = 0.3549107142857143


-----


# 회고!


- 요즘은 모델을 처음부터 짜는게 아니라, 이미 잘 만들어진 모델을 가져와 내 입맛에 맞게 커스텀하고, 필요에 맞게 개량한다 라는 말은 자주 들었었다. hugging face 처럼 사람들이 공유한 것을 가져와 사용해 보거나, 간편하게 공유해서 올리는 등의 웹페이지를 알게되서 기쁘다. 확실하게 이해하고 넘어간 것이 아니라, 그저 노드와 다른분의 글을 참조해 따라가기 급급해서 정말 대략적인것만 알고 넘어갔기 때문에, 추후에 애프터서비스가 확실하게 필요 할 것이다. 꼭 이쪽으로 가지 않더라도 분명 필요한 순간이 올테니 꼭 기억해 뒀다가, 필요해 졌을때 유용하게 써먹을 수 있게 열심히 노력해야겠다:) 
- 아니 근데 왜 에러가....... 

- 이제 아이펠 과정에선 약 6주간의 큰 아이펠톤만 남겨놓고 있다. 여태까지 했던것도 노션에 정리해서 이런과정을 거쳐 내가 되었다 라는 포트폴리오를 슬슬 준비 해야할 것 같다. 
- 여태까지 고생 많았고, 남은 기간 동안 팀원들과 좋은 성적을 거둘 수 있도록 열심히 노력해야지! 
- 물론 취업활동도 열심히 하구 ^^; 

# 참고 페이지

- [이전 기수분의 소중한 프로젝트 데이터](https://github.com/ik-e/going_deeper/blob/ed5935fb80890fd9769487bf0987b91952a8de7d/%5BNLP-18%5Dhuggingface.ipynb)

- [hugging Face 페이지](https://huggingface.co/models)

- [transformers 공식 github](https://github.com/huggingface/transformers)